In [1]:
import json
import polars as pl
import pandas as pd
import numpy as np
import tqdm
import requests
from bs4 import BeautifulSoup
import time

from pandarallel import pandarallel
pandarallel.initialize(
    progress_bar=True,
    nb_workers=16,
)

from transformers import pipeline

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
data = pl.read_parquet('../data/hh_recsys_vacancies.pq').to_pandas()
data.head(2)

,vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
0,v_862116,Смотритель музейный,c_162972,<strong>Обязанности:</strong> <ul> <li>Осущест...,"[Пользователь ПК, Работа в команде, Умение пла...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"<p><strong>Возможно, наша вакансия именно та, ...","[Активные продажи, Холодные продажи, Кредитные...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience


In [3]:
np.percentile(data['description'].str.len(), [0.5, 0.9, 0.99, 0.999, 0.9999])

array([281., 307., 312., 313., 313.])

In [6]:
def get_text(row):
    skills = row['keySkills.keySkill']
    skills = skills.tolist() if isinstance(skills, list) else []
    text = '. '.join(
        [
            row['name'],
            BeautifulSoup(row['description']).get_text(),
        ] + skills,
    )[:1024]
    return text.strip()

get_text(data.iloc[2])

'Бухгалтер (по расчету зарплаты). Обязанности:  Расчет заработной платы (в том числе вахтовый метод), больничных листов, командировочных, пособий и прочих выплат персоналу (суммированный учет) 300 человек Расчет по исполнительным документам Расчет налогов Составление и сдача отчетности в ИФНС, ФСС, ПФР Соблюдение законодательства  Требования:  Высшее образование (экономическое, бухучет/аудит) Опыт работы на данном участке от 3-х лет Знание бухгалтерского и налогового учета выплат персоналу   Знание законодательства: 255-ФЗ, ТК РФ и НК РФ в части расчета заработной платы Уверенный пользователь ПК: 1С Предприятие 8.3, ЗУП; Внимательность, ответственность, умение работать с большим объемом документов.  Условия:  Оформление в соответствии с ТК РФ. Полный рабочий день, 9.00-18.00, пятидневка. Стабильная и своевременная оплата труда.'

In [7]:
data['text'] = data.parallel_apply(get_text, axis='columns')
data.head(3)

/tmp/ipykernel_3688/3530420921.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  BeautifulSoup(row['description']).get_text(),
/tmp/ipykernel_3688/3530420921.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  BeautifulSoup(row['description']).get_text(),
/tmp/ipykernel_3688/3530420921.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  BeautifulSoup(row['description']).get_text(),
/tmp/ipykernel_3688/3530420921.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  BeautifulSoup(row['description']).get_text(),
/tmp/ipykernel_3688/3530420921.p

,vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,text
0,v_862116,Смотритель музейный,c_162972,<strong>Обязанности:</strong> <ul> <li>Осущест...,"[Пользователь ПК, Работа в команде, Умение пла...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Смотритель музейный. Обязанности: Осуществлят...
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"<p><strong>Возможно, наша вакансия именно та, ...","[Активные продажи, Холодные продажи, Кредитные...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Ведущий менеджер по работе с физическими лицам...
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,<strong>Обязанности:</strong> <ul> <li>Расчет ...,None,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6,Бухгалтер (по расчету зарплаты). Обязанности: ...


In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('cointegrated/rubert-tiny2')
sentences = ["привет мир", "hello world", "здравствуй вселенная"]
embeddings = model.encode(sentences)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
np.array(embeddings).shape

(3, 312)

In [13]:
parts = []

for chunk in tqdm.tqdm(np.array_split(data, data.shape[0] // 1000)):
    r = model.encode(chunk['text'].tolist())
    parts.append(np.array(r))


/home/roma1n/.cache/pypoetry/virtualenvs/hh-6r2Tdj88-py3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
  0%|                           | 2/2734 [00:26<10:11:00, 13.42s/it]


KeyboardInterrupt: 

In [17]:
full = np.concatenate(parts)

In [25]:
full.shape

(2002, 312)

In [26]:
data.head()

,vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,text
0,v_862116,Смотритель музейный,c_162972,<strong>Обязанности:</strong> <ul> <li>Осущест...,"[Пользователь ПК, Работа в команде, Умение пла...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Смотритель музейный. Обязанности: Осуществлят...
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"<p><strong>Возможно, наша вакансия именно та, ...","[Активные продажи, Холодные продажи, Кредитные...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Ведущий менеджер по работе с физическими лицам...
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,<strong>Обязанности:</strong> <ul> <li>Расчет ...,None,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6,Бухгалтер (по расчету зарплаты). Обязанности: ...
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,"<p><strong>Для каждого, кто хочет работать и з...",None,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience,"Пекарь (Токсово, Привокзальная, 16). Для каждо..."
4,v_312507,Торговый представитель (г. Абакан),c_206699,<p>Компания ТД &quot;Оливьера&quot;- лучший ди...,"[Продуктивность, Клиентоориентированность, Упр...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Торговый представитель (г. Абакан). Компания Т...


In [19]:
np.savez('../data/vac_text.npz', full)

In [20]:
x = np.load('../data/vac_text.npz')

In [24]:
(x['arr_0'] == full).all()

True